In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [3]:
df_fake = pd.read_csv("../input/fake-news-detection/Fake.csv")
df_true = pd.read_csv("../input/fake-news-detection/True.csv")

In [4]:
df_fake.head()

In [5]:
df_true.head(5)

In [6]:
df_fake.shape, df_true.shape

In [7]:
df_fake["class"] = 0
df_true["class"] = 1

In [8]:
df_fake

In [9]:
df_fake.shape, df_true.shape

In [10]:
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head(10)

In [11]:
df_merge.columns

In [12]:
df = df_merge.drop(["title", "subject","date"], axis = 1)

In [13]:
df.head(10)

In [14]:
df.isnull().sum()

In [15]:
df.shape

In [16]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [17]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    filtered_sentence = ''

    for w in word_tokens:
        if w not in stop_words and len(filtered_sentence) == 0:
            filtered_sentence += w
        elif w not in stop_words and len(filtered_sentence) != 0:
            filtered_sentence += ' '+w
    
    
    return filtered_sentence

In [18]:
df["text"] = df["text"].apply(wordopt)

In [19]:
x = df["text"]
y = df["class"]

In [20]:
x.count()

In [21]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [23]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train,y_train)
pred_lr=LR.predict(xv_test)

In [24]:
print(classification_report(y_test, pred_lr))

In [25]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)
pred_dt = DT.predict(xv_test)

In [26]:
print(classification_report(y_test, pred_dt))

In [27]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)
pred_gbc = GBC.predict(xv_test)

In [28]:
print(classification_report(y_test, pred_gbc))

In [29]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)
pred_rfc = RFC.predict(xv_test)

In [30]:
print(classification_report(y_test, pred_rfc))

In [31]:
pip install xgboost

In [32]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(xv_train, y_train)
pred_xgb = xgb.predict(xv_test)

In [33]:
print(classification_report(y_test, pred_xgb))

In [34]:
import lightgbm as lgb
LGB = lgb.LGBMClassifier()
LGB.fit(xv_train, y_train)
pred_lgb=LGB.predict(xv_test)

In [35]:
print(classification_report(y_test, pred_lgb))

In [36]:
vectorization_all = TfidfVectorizer()
xv_train_all = vectorization_all.fit_transform(x)
##############################################################################################################
LR_all = LogisticRegression()
LR_all.fit(xv_train_all,y)
##############################################################################################################
DT_all = DecisionTreeClassifier()
DT_all.fit(xv_train_all,y)
##############################################################################################################
GBC_all = GradientBoostingClassifier(random_state=0)
GBC_all.fit(xv_train_all,y)
##############################################################################################################
RFC_all = RandomForestClassifier(random_state=0)
RFC_all.fit(xv_train_all,y)
##############################################################################################################
xgb_all = XGBClassifier()
xgb_all.fit(xv_train_all,y)
##############################################################################################################
LGB_all = lgb.LGBMClassifier()
LGB_all.fit(xv_train_all,y)

In [37]:
# import joblib

In [38]:
# joblib.dump(vectorization_all, 'vectorization.pkl')

In [39]:
# joblib.dump(LR_all, 'LR.pkl')
# joblib.dump(DT_all, 'DT.pkl')
# joblib.dump(GBC_all, 'GBC.pkl')
# joblib.dump(RFC_all, 'RFC.pkl')
# joblib.dump(xgb_all, 'xgb.pkl')
# joblib.dump(LGB_all, 'LGB.pkl')

In [40]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization_all.transform(new_x_test)
    pred_LR = LR_all.predict(new_xv_test)
    pred_DT = DT_all.predict(new_xv_test)
    pred_GBC = GBC_all.predict(new_xv_test)
    pred_RFC = RFC_all.predict(new_xv_test)
    pred_xgb = xgb_all.predict(new_xv_test)
    pred_lgb = LGB_all.predict(new_xv_test)
    print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {} \nxgb Prediction: {} \nLBG Prediction: {}".format(output_lable(pred_LR[0]),
                                                                                                              output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0]),
                                                                                                              output_lable(pred_xgb[0]),
                                                                                                              output_lable(pred_lgb[0])))
    print("#-------------------------------------------------------------------------------")

In [41]:
df_testvikas = pd.read_csv("../input/testvikas12/NLP_DATA - vikas.csv")

In [42]:
test_data = pd.DataFrame(df_testvikas['E'])

In [43]:
test_data['E'] =test_data['E'].apply(wordopt)

In [44]:
rows_test_data = test_data.count()[0]

In [45]:
for i in range(10):
    news = test_data['E'][i]
    manual_testing(news)

In [46]:
from lime.lime_text import LimeTextExplainer
from sklearn.pipeline import make_pipeline
class_names=['non authentic','authentic']
explainer = LimeTextExplainer(class_names=class_names)
tfidf  = vectorization_all

In [47]:
def limetesting(idx):
    print(idx)
    print('#LR------------------------------------------------------------------------------')
    
    c_lr = make_pipeline(tfidf, LR_all)
    exp_lr = explainer.explain_instance(test_data.E[idx], c_lr.predict_proba, num_features=10)
    exp_lr.show_in_notebook(text=True)
    
    print('#DT------------------------------------------------------------------------------')
    
    
    c_dt = make_pipeline(tfidf, DT_all)
    exp_dt = explainer.explain_instance(test_data.E[idx], c_dt.predict_proba, num_features=10)
    exp_dt.show_in_notebook(text=True)
    
    print('#GBC------------------------------------------------------------------------------')
    
    
    c_gbc = make_pipeline(tfidf, GBC_all)
    exp_gbc = explainer.explain_instance(test_data.E[idx], c_gbc.predict_proba, num_features=10)
    exp_gbc.show_in_notebook(text=True)
    
    print('#RFC------------------------------------------------------------------------------')
    
    
    c_rfc = make_pipeline(tfidf, RFC_all)
    exp_rfc = explainer.explain_instance(test_data.E[idx], c_rfc.predict_proba, num_features=10)
    exp_rfc.show_in_notebook(text=True)
    
    print('#XGB------------------------------------------------------------------------------')
    
    c_xgb = make_pipeline(tfidf, xgb_all)
    exp_xgb = explainer.explain_instance(test_data.E[idx], c_xgb.predict_proba, num_features=10)
    exp_xgb.show_in_notebook(text=True)
    
    print('#LGB------------------------------------------------------------------------------')
    
    c_lgb = make_pipeline(tfidf, LGB_all)
    exp_lgb = explainer.explain_instance(test_data.E[idx], c_lgb.predict_proba, num_features=10)
    exp_lgb.show_in_notebook(text=True)
    
    
    

In [48]:
for i in range(10):
    limetesting(i)